In [116]:
# Load the input
with open("day1_input.txt") as f:
    day1_input = f.read()

print(day1_input)

6798seven
six8b32csscsdgjsevenfivedlhzhc
fcvvkvjjnhfivesixfiveninesxlvnhfckc54
feightwo4twofivefour
cgpzm2sevenone68636
498879
fivetwo2plltvdfourhhgrfn
onertxzlzdnxn2seven
ninethree4v1five
5pnnss16lzbsmcckeightdseven
jthree249four2cjzqmmbrqnnine
foursixfourfbgvqm1rchdlmzhtzqqfjzpmln
twojtwoscbrtgmnqzvbfbzvrtkxhmf76nine
2fourtwo
svxz8bjvt5zzvjzhkmtmnnft92
threetffnjvkfh81nineqqshzdf2vtjfqjtbdv
seven1drfivethreedpqflxgdshlzmmnrb
7sggjxkcxfourthreefourtwos3nine
39t6rvkbdnfpssj6djnfbqklkgfsqbf
3four2zcfvtplkrbeight274
one7sixninesix
kkxmtmdthree6jrj6
81sevenmnine1llbqrsprc
5nine9qgjceight
three1sdmq9sevenfournine
khnlbmzhvlsix3
nineone6onesixvlnlxeightfive
schplznseveneightnlcxgr7four
6one1djcdmpdrgq3two
fourzhpnphmq52r813four
scsxjthreefoureight2fivepfmpfj8
69xvbxfffmr7one8gmch8
one8tfpgrdhm
2gxmhtfrbrkpdvlvsmdqrktxtrpssbrv933
4vfrtkdqlbtwordlbppsssp
2cjmtvnzpbkdcq5twofourtwo
2three5onetwogpfqtszbjh
hgvnine9four996
37142745
four73
8552ztclnd
onevmpd76eighth
threethree8
bgtwonedrmc35
23one

In [19]:
# Part 1: brute force
line_vals = []
for line in day1_input.split("\n"):
    first_numeric = last_numeric = None
    for char_idx in range(len(line)):
        if first_numeric is None and line[char_idx].isnumeric():
            first_numeric = line[char_idx]
        if last_numeric is None and line[-(char_idx+1)].isnumeric():
            last_numeric = line[-(char_idx+1)]
    
    line_val = int(first_numeric + last_numeric)
    line_vals.append(line_val)
    # print(f"{line}\t{first_numeric}\t{last_numeric}\t{line_val}")

6798seven	6	8	68
six8b32csscsdgjsevenfivedlhzhc	8	2	82
fcvvkvjjnhfivesixfiveninesxlvnhfckc54	5	4	54
feightwo4twofivefour	4	4	44
cgpzm2sevenone68636	2	6	26
498879	4	9	49
fivetwo2plltvdfourhhgrfn	2	2	22
onertxzlzdnxn2seven	2	2	22
ninethree4v1five	4	1	41
5pnnss16lzbsmcckeightdseven	5	6	56
jthree249four2cjzqmmbrqnnine	2	2	22
foursixfourfbgvqm1rchdlmzhtzqqfjzpmln	1	1	11
twojtwoscbrtgmnqzvbfbzvrtkxhmf76nine	7	6	76
2fourtwo	2	2	22
svxz8bjvt5zzvjzhkmtmnnft92	8	2	82
threetffnjvkfh81nineqqshzdf2vtjfqjtbdv	8	2	82
seven1drfivethreedpqflxgdshlzmmnrb	1	1	11
7sggjxkcxfourthreefourtwos3nine	7	3	73
39t6rvkbdnfpssj6djnfbqklkgfsqbf	3	6	36
3four2zcfvtplkrbeight274	3	4	34
one7sixninesix	7	7	77
kkxmtmdthree6jrj6	6	6	66
81sevenmnine1llbqrsprc	8	1	81
5nine9qgjceight	5	9	59
three1sdmq9sevenfournine	1	9	19
khnlbmzhvlsix3	3	3	33
nineone6onesixvlnlxeightfive	6	6	66
schplznseveneightnlcxgr7four	7	7	77
6one1djcdmpdrgq3two	6	3	63
fourzhpnphmq52r813four	5	3	53
scsxjthreefoureight2fivepfmpfj8	2	8	28
69xvbxfffmr7one8gm

In [20]:
print(f"Answer (part one): {sum(line_vals)}")

Answer (part one): 54239


In [105]:
# Part 2 - data
nums_as_strings = {
    "one":   1,
    "two":   2,
    "three": 3,
    "four":  4,
    "five":  5,
    "six":   6,
    "seven": 7,
    "eight": 8,
    "nine":  9,
}
max_num_len = max([len(x) for x in nums_as_strings.keys()])

# Array of sets, set at idx 0 contains the valid 1 char starts, 1 the 2 chars etc.
valid_starts = []
valid_ends = []
for i in range(max_num_len+1):
    valid_starts.append(set([nas[0:i] if len(nas) >= i else None for nas in nums_as_strings.keys()]))
    valid_ends.append(set([nas[len(nas)-i:] if len(nas) >= i else None for nas in nums_as_strings.keys()]))

In [120]:
# Left to right iterator test
# i is the marker for the left hand side of our search frame, j is the right
# if i == j then the character at target[i] is what we're looking at
# i can never be greater than j
# for every slice target[i:i] to target[i:j], we should check if that slice represents a number
# if the slice target[i:n] is not a valid start for that n (i.e. n=2 and target[1:n] is a 2 char
# string which isn't a valid start of a word-number) then we move i up by 1

# Reverse mode is the same logic but i / j start at len() and move down to 0 
def get_first_num_in_string(target: str, reverse=False) -> int:
    target_len = len(target)
    def move_marker(x):
        return x + 1
    def check_marker(x):
        return x < target_len
    def reset_i():
        return 0
    def reset_j(i):
        return i + 1
    def get_frame(i,j):
        if reverse:
            return target[target_len-j:target_len-i]
        else:
            return target[i:j]
    def check_frame_validity(frame):
        if reverse:
            return frame in valid_ends[frame_size]
        else:
            return frame in valid_starts[frame_size]

        
    i = reset_i()
    
    result = None
    # Expecting one, eight
    # Don't forget to check actual numerics!
    while check_marker(i) and result is None:
        j = reset_j(i)

        # print(f"i: {i}")
        # print(f"j: {j}")

        # Check for an actual number
        frame = get_frame(i,j)
        # print(f"Checking frame (numeric): {frame}")
        if frame.isnumeric():
            result = int(frame)
            # print(f"Found a number! {result}")
            break

        # print(f"check_marker(j): {check_marker(j)}")
        while check_marker(j):
            frame_size = j - i
            frame = get_frame(i,j)
            # print(f"Checking frame (string): {frame}")
            if not check_frame_validity(frame):
                # print(f"Invalid frame start, skipping... {frame}")
                break

            # print(f"Checking frame: {frame}")
            if frame in nums_as_strings:
                result = nums_as_strings[frame]
                # print(f"Found a number! {frame} => {result}")
                break

            j = move_marker(j)
        
        i = move_marker(i)
    
    return result

In [112]:
target = day1_input.split("\n")[2]
print(target)
get_first_num_in_string(target = target,reverse=False)
# get_first_num_in_string(target = day1_input.split("\n")[0],reverse=False)

fcvvkvjjnhfivesixfiveninesxlvnhfckc54


5

In [121]:
# Part 2
line_vals = []
for line in day1_input.split("\n"):
    first_number = get_first_num_in_string(target = line,reverse=False)
    last_number = get_first_num_in_string(target = line,reverse=True)

    if first_number is None or last_number is None:
        raise Exception(f"Found no numbers in the string! ({line})")
    
    line_val = first_number*10 + last_number
    line_vals.append(line_val)
    print(f"{line}\t{first_number}\t{last_number}\t{line_val}")

print(f"Answer (part two): {sum(line_vals)}")

6798seven	6	7	67
six8b32csscsdgjsevenfivedlhzhc	6	5	65
fcvvkvjjnhfivesixfiveninesxlvnhfckc54	5	4	54
feightwo4twofivefour	8	4	84
cgpzm2sevenone68636	2	6	26
498879	4	9	49
fivetwo2plltvdfourhhgrfn	5	4	54
onertxzlzdnxn2seven	1	7	17
ninethree4v1five	9	5	95
5pnnss16lzbsmcckeightdseven	5	7	57
jthree249four2cjzqmmbrqnnine	3	9	39
foursixfourfbgvqm1rchdlmzhtzqqfjzpmln	4	1	41
twojtwoscbrtgmnqzvbfbzvrtkxhmf76nine	2	9	29
2fourtwo	2	2	22
svxz8bjvt5zzvjzhkmtmnnft92	8	2	82
threetffnjvkfh81nineqqshzdf2vtjfqjtbdv	3	2	32
seven1drfivethreedpqflxgdshlzmmnrb	7	3	73
7sggjxkcxfourthreefourtwos3nine	7	9	79
39t6rvkbdnfpssj6djnfbqklkgfsqbf	3	6	36
3four2zcfvtplkrbeight274	3	4	34
one7sixninesix	1	6	16
kkxmtmdthree6jrj6	3	6	36
81sevenmnine1llbqrsprc	8	1	81
5nine9qgjceight	5	8	58
three1sdmq9sevenfournine	3	9	39
khnlbmzhvlsix3	6	3	63
nineone6onesixvlnlxeightfive	9	5	95
schplznseveneightnlcxgr7four	7	4	74
6one1djcdmpdrgq3two	6	2	62
fourzhpnphmq52r813four	4	4	44
scsxjthreefoureight2fivepfmpfj8	3	8	38
69xvbxfffmr7one8gm